In [24]:
import sqlite3
import pandas as pd

In [13]:
conn = sqlite3.connect('sqlite_db_pythonsqlite.db')

In [14]:
cur = conn.cursor()

In [100]:
conn.row_factory = sqlite3.Row 

In [54]:
q1 = '''SELECT * FROM FACILITIES'''

In [55]:
cur.execute(q1)

In [56]:
cur.fetchall()

[(0, 'Tennis Court 1', 5, 25, 10000, 200),
 (1, 'Tennis Court 2', 5, 25, 8000, 200),
 (2, 'Badminton Court', 0, 15.5, 4000, 50),
 (3, 'Table Tennis', 0, 5, 320, 10),
 (4, 'Massage Room 1', 9.9, 80, 4000, 3000),
 (5, 'Massage Room 2', 9.9, 80, 4000, 3000),
 (6, 'Squash Court', 3.5, 17.5, 5000, 80),
 (7, 'Snooker Table', 0, 5, 450, 15),
 (8, 'Pool Table', 0, 5, 400, 15)]

In [98]:
def Query(query):
    cur = conn.cursor()
    cur.execute(query)
    rows = cur.fetchall()
    helper = []
    for row in rows:
        helper.append(dict(row))
    rez = pd.DataFrame(helper)
    return rez

In [108]:
Query('SELECT * FROM FACILITIES ORDER BY membercost')

,facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
0,2,Badminton Court,0.0,15.5,4000,50
1,3,Table Tennis,0.0,5.0,320,10
2,7,Snooker Table,0.0,5.0,450,15
3,8,Pool Table,0.0,5.0,400,15
4,6,Squash Court,3.5,17.5,5000,80
5,0,Tennis Court 1,5.0,25.0,10000,200
6,1,Tennis Court 2,5.0,25.0,8000,200
7,4,Massage Room 1,9.9,80.0,4000,3000
8,5,Massage Room 2,9.9,80.0,4000,3000


In [111]:
Query("PRAGMA table_info(FACILITIES)").T

,0,1,2,3,4,5
cid,0,1,2,3,4,5
name,facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
type,int(1),varchar(15),"decimal(2,1)","decimal(3,1)",int(5),int(4)
notnull,1,0,0,0,0,0
dflt_value,'0',NULL,NULL,NULL,NULL,NULL
pk,1,0,0,0,0,0


In [105]:
Query("PRAGMA table_info(MEMBERS)").T

,0,1,2,3,4,5,6,7
cid,0,1,2,3,4,5,6,7
name,memid,surname,firstname,address,zipcode,telephone,recommendedby,joindate
type,int(2),varchar(17),varchar(9),varchar(39),int(5),varchar(14),varchar(2),varchar(19)
notnull,1,0,0,0,0,0,0,0
dflt_value,'0',NULL,NULL,NULL,NULL,NULL,NULL,NULL
pk,1,0,0,0,0,0,0,0


In [106]:
Query("PRAGMA table_info(BOOKINGS)").T

,0,1,2,3,4
cid,0,1,2,3,4
name,bookid,facid,memid,starttime,slots
type,int(4),int(1),int(2),varchar(19),int(2)
notnull,1,0,0,0,0
dflt_value,'0',NULL,NULL,NULL,NULL
pk,1,0,0,0,0


Q10: Produce a list of facilities with a total revenue less than 1000.
The output of facility name and total revenue, sorted by revenue. Remember
that there's a different cost for guests and members! 

In [114]:
Query("""
SELECT f.name, 
CASE WHEN m.memid > 0 THEN SUM(b.slots * f.membercost) ELSE SUM(b.slots * f.guestcost) END AS revenue
FROM BOOKINGS AS b
LEFT JOIN FACILITIES AS f ON b.facid = f.facid
LEFT JOIN MEMBERS AS m ON b.memid = m.memid
GROUP BY f.name
HAVING revenue < 1000
ORDER BY revenue
""")

,name,revenue
0,Badminton Court,0
1,Pool Table,0
2,Snooker Table,0
3,Table Tennis,0


Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order 

In [117]:
Query("""SELECT ( m.surname || ', ' || m.firstname ) AS MemName, ( c.surname || ', ' || c.firstname ) AS Recommender
FROM Members AS m
INNER JOIN Members AS c ON m.recommendedby = c.memid
WHERE m.recommendedby >=1
ORDER BY MemName
LIMIT 0 , 30""")

,MemName,Recommender
0,"Bader, Florence","Stibbons, Ponder"
1,"Baker, Anne","Stibbons, Ponder"
2,"Baker, Timothy","Farrell, Jemima"
3,"Boothe, Tim","Rownam, Tim"
4,"Butters, Gerald","Smith, Darren"
5,"Coplin, Joan","Baker, Timothy"
6,"Crumpet, Erica","Smith, Tracy"
7,"Dare, Nancy","Joplette, Janice"
8,"Genting, Matthew","Butters, Gerald"
9,"Hunt, John","Purview, Millicent"


Q12: Find the facilities with their usage by member, but not guests

In [120]:
Query("""SELECT f.name,
CASE WHEN m.memid =0
THEN SUM( b.slots * f.guestcost )
ELSE ''
END AS GuestRev
FROM Bookings AS b
LEFT JOIN Facilities AS f ON b.facid = f.facid
LEFT JOIN Members AS m ON b.memid = m.memid
GROUP BY f.name
ORDER BY GuestRev DESC
LIMIT 0 , 30""")

,name,GuestRev
0,Tennis Court 1,
1,Table Tennis,
2,Snooker Table,
3,Pool Table,
4,Massage Room 1,
5,Badminton Court,
6,Tennis Court 2,31950
7,Squash Court,19320
8,Massage Room 2,18240


Q13: Find the facilities usage by month, but not guests

In [143]:
Query("""
SELECT facid, 
SUM(slots) AS Usage, 
strftime('%m', starttime) AS Month 
FROM Bookings 
WHERE memid <> 0
GROUP BY facid, strftime('%m', starttime)
""")

,facid,Usage,Month
0,0,201,07
1,0,339,08
2,0,417,09
3,1,123,07
4,1,345,08
5,1,414,09
6,2,165,07
7,2,414,08
8,2,507,09
9,3,98,07
